In [ ]:
%run "../scripts/script_abl_vs_enz_use_heatmap.py"

In [ ]:
# Initialise model
glc_exch_rate = 16.89
wt_ec = Yeast8Model("../data/gemfiles/ecYeastGEM_batch_8-6-0.xml")
wt_ec.model.reactions.get_by_id("r_1714").bounds = (-glc_exch_rate, 0)
wt_ec.model.reactions.get_by_id("r_1714_REV").bounds = (0, glc_exch_rate)

# Ablate and store fluxes in each round
wt_ec.ablation_result = wt_ec.ablate()
ablation_fluxes = wt_ec.ablation_fluxes

ablation_fluxes_diff = ablation_fluxes.copy()
ablation_fluxes_diff.pop("original")
for biomass_component, fluxes in ablation_fluxes_diff.items():
    ablation_fluxes_diff[biomass_component] = (
        ablation_fluxes[biomass_component] - ablation_fluxes["original"]
    )

In [ ]:
def flux_dict_to_df(flux_dict):
    list_participating_rxn_df = []
    # Make DF for each biomass component
    for idx, (biomass_component, fluxes) in enumerate(flux_dict.items()):
        # get fluxes
        s = fluxes.copy()
        # get data needed for DF
        enz_metabolite_ids = extract_protein_ids(s)
        participating_rxn_ids, enz_usage_fluxes = get_participating_rxn_ids(
            enz_metabolite_ids, s, wt_ec
        )
        subsystem_list = get_subsystem_list(participating_rxn_ids, subsystem_dict)
        # construct DF
        enz_usage_flux_column = "enz_usage_flux_" + biomass_component
        participating_rxn_df = pd.DataFrame(
            {
                "participating_rxn_id": participating_rxn_ids,
                "subsystem": subsystem_list,
                enz_usage_flux_column: enz_usage_fluxes,
            }
        )
        list_participating_rxn_df.append(participating_rxn_df)
    # construct master DF with info from all biomass components
    left_columns = list_participating_rxn_df[0].iloc[:, 0:2]
    enz_usage_columns = pd.concat(
        [
            list_participating_rxn_df[idx].iloc[:, -1]
            for idx in range(len(list_participating_rxn_df))
        ],
        axis=1,
    )
    
    all_fluxes_df = pd.concat([left_columns, enz_usage_columns], axis=1)
    # Drop enzyme usage reactions
    all_fluxes_df = all_fluxes_df[all_fluxes_df.subsystem != "Enzyme usage"]
    # Deal with duplicate reaction IDs by summing the fluxes
    all_fluxes_df = all_fluxes_df.groupby(
        ["participating_rxn_id", "subsystem"], as_index=False
    ).sum(numeric_only=True)
    # Sort alphabetically by subsystem
    all_fluxes_df = all_fluxes_df.sort_values(by=["subsystem"])
    
    return all_fluxes_df

In [ ]:
d = flux_dict_to_df(ablation_fluxes_diff)

In [ ]:
d

In [ ]:
f = flux_dict_to_df(ablation_fluxes)

In [ ]:
f

# Zero original flux -- differences

In [ ]:
zero_orig_flux_df = f[f["enz_usage_flux_original"] == 0]

In [ ]:
zero_orig_flux_df

In [ ]:
# Drop fluxes with magnitude below a certain tol value
enz_usage_colnames = [
    colname
    for colname in zero_orig_flux_df.columns.to_list()
    if colname.startswith("enz_usage_flux_")
]
enz_usage_colnames.remove("enz_usage_flux_original")
tol = plot_options["subsystem_heatmap/tol"]
zero_orig_flux_df = f.loc[
    (f[enz_usage_colnames].abs() >= tol).any(axis=1)
]
print(f"heatmap has {len(zero_orig_flux_df)} rows")

In [ ]:
# Define y-labels for heatmap: subsystem names
subsystem_list_sorted = zero_orig_flux_df.subsystem.to_list()
# Replace duplicates with space so that the subsystem name is only
# present the first time it occurs
subsystem_labels = []
compare_string = ""
for subsystem_string in subsystem_list_sorted:
    if subsystem_string == compare_string:
        subsystem_string = " "
    else:
        compare_string = subsystem_string
    subsystem_labels.append(subsystem_string)

In [ ]:
# Draw 2d heatmap
fig, ax = plt.subplots(figsize=(10, 25))
sns.heatmap(
    zero_orig_flux_df.iloc[:, 3:] * 1e4,
    xticklabels=list(ablation_fluxes_diff.keys()),
    yticklabels=subsystem_labels,
    center=0,
    robust=True,
    cmap="PiYG",
    cbar_kws={
        "label": r"Flux change [$\times 10^{-4} \mathrm{protein} mmol \cdot g_{DW}^{-1}$]"
    },
    ax=ax,
)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_title(
    f"Enzyme usage changes as a result of ablation\n(magnitudes smaller than {tol} excluded)"
)

# Non-zero original fluxes -- fold changes

In [ ]:
nonzero_orig_flux_df = f[f["enz_usage_flux_original"] != 0]

In [ ]:
nonzero_orig_flux_df

In [ ]:
enz_usage_colnames = [
    colname
    for colname in nonzero_orig_flux_df.columns.to_list()
    if colname.startswith("enz_usage_flux_")
]
enz_usage_colnames.remove("enz_usage_flux_original")

In [ ]:
nonzero_foldchanges = nonzero_orig_flux_df.copy()

In [ ]:
for colname in enz_usage_colnames:
    nonzero_foldchanges[colname] /= nonzero_foldchanges["enz_usage_flux_original"]
    nonzero_foldchanges[colname] = np.log2(nonzero_foldchanges[colname])
nonzero_foldchanges.drop(columns=["enz_usage_flux_original"], inplace=True)
# log(0) is undefined
nonzero_foldchanges.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
nonzero_foldchanges

In [ ]:
# Define y-labels for heatmap: subsystem names
subsystem_list_sorted = nonzero_foldchanges.subsystem.to_list()
# Replace duplicates with space so that the subsystem name is only
# present the first time it occurs
subsystem_labels = []
compare_string = ""
for subsystem_string in subsystem_list_sorted:
    if subsystem_string == compare_string:
        subsystem_string = " "
    else:
        compare_string = subsystem_string
    subsystem_labels.append(subsystem_string)

In [ ]:
# Draw 2d heatmap
fig, ax = plt.subplots(figsize=(10, 25))
sns.heatmap(
    nonzero_foldchanges.iloc[:, 2:],
    xticklabels=list(ablation_fluxes_diff.keys()),
    yticklabels=subsystem_labels,
    center=0,
    robust=True,
    cmap="PiYG",
    cbar_kws={
        "label": r"log2(Fold change)"
    },
    ax=ax,
)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_title(
    f"Enzyme usage fold changes as a result of ablation"
)

In [ ]:
df_nrows = len(nonzero_foldchanges)
third = int(np.floor(df_nrows/3))
twothirds = int(np.floor(2*df_nrows/3))

In [ ]:
# First third
fig, ax = plt.subplots(figsize=(10, 25))
sns.heatmap(
    nonzero_foldchanges.iloc[0:third, 2:],
    xticklabels=list(ablation_fluxes_diff.keys()),
    yticklabels=subsystem_labels[0:third],
    center=0,
    robust=True,
    cmap="PiYG",
    cbar_kws={
        "label": r"log2(Fold change)"
    },
    ax=ax,
)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_title(
    f"Enzyme usage fold changes as a result of ablation"
)

In [ ]:
# Second third
fig, ax = plt.subplots(figsize=(10, 25))
sns.heatmap(
    nonzero_foldchanges.iloc[third+1:twothirds, 2:],
    xticklabels=list(ablation_fluxes_diff.keys()),
    yticklabels=subsystem_labels[third+1:twothirds],
    center=0,
    robust=True,
    cmap="PiYG",
    cbar_kws={
        "label": r"log2(Fold change)"
    },
    ax=ax,
)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_title(
    f"Enzyme usage fold changes as a result of ablation"
)

In [ ]:
# Last third
fig, ax = plt.subplots(figsize=(10, 25))
sns.heatmap(
    nonzero_foldchanges.iloc[twothirds+1:, 2:],
    xticklabels=list(ablation_fluxes_diff.keys()),
    yticklabels=subsystem_labels[twothirds+1:],
    center=0,
    robust=True,
    cmap="PiYG",
    cbar_kws={
        "label": r"log2(Fold change)"
    },
    ax=ax,
)
ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
ax.set_title(
    f"Enzyme usage fold changes as a result of ablation"
)